In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmread
#X = csr_matrix(mmread('test.x.mm'))
#y = np.loadtxt('test.y.dat')

print "hola.. estamos realizando la operacion"

#para los datos con columna oscar
X_test = csr_matrix(mmread('ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/test.x.mm'))
y_test = np.loadtxt('ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/test.y.dat')

X_train = csr_matrix(mmread('ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/train.x.mm'))
y_train = np.loadtxt('ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/train.y.dat')

X_val = csr_matrix(mmread('ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/dev.x.mm'))
y_val = np.loadtxt('ngrams-deprels-fp1-origin.runtime.budget.numscreen.ratings.seasons.stars/dev.y.dat')

print "termino de cargar la data con menos caracteristicas (ya filtrada)"

import sklearn.linear_model as lm
def MSE(y,yhat): return np.mean(np.power(y-yhat,2))

def best_alpha(alphas_, X,y, model, X_val,y_val):
    best_cv_mse = float("inf")
    for a in alphas_:
        model.set_params(alpha=a)
        mse =  MSE(model.fit(X,y).predict(X_val) , y_val)
        if mse < best_cv_mse:
            best_cv_mse = mse
            best_alpha = a
    return best_alpha

model = lm.LinearRegression(fit_intercept = False)
model.fit(X_train,y_train)
best_score = model.score(X_test,y_test)
print "Score de regresion ordinaria: %f"%(best_score)

model = lm.Ridge(fit_intercept = False  )
alphas_ = np.logspace(2,5, base = 10)
a = best_alpha(alphas_,X_train,y_train,model,X_val,y_val)
model.set_params(alpha=a)
model.fit(X_train,y_train)
score = model.score(X_test, y_test)
print "Score de regresion Ridge: %f , con alpha: %f"%(score,a)

model = lm.Lasso(fit_intercept = False)
alphas_ = np.logspace(8,4, base= 10)
a =  255954.792270 #best_alpha(alphas_, X_train,y_train, model, X_val,y_val)  #a = 255954.792270 
model.set_params(alpha=a)
model.fit(X_train,y_train)
score = model.score(X_test, y_test)
print "Score de regresion Lasso: %f , con alpha: %f"%(score,a)

hola.. estamos realizando la operacion
termino de cargar la data con menos caracteristicas (ya filtrada)
Score de regresion ordinaria: 0.590312
Score de regresion Ridge: 0.591824 , con alpha: 1676.832937
Score de regresion Lasso: 0.577160 , con alpha: 255954.792270


/usr/lib/python2.7/dist-packages/sklearn/linear_model/coordinate_descent.py:490: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


Analisis de error para Ridge

In [4]:
alphas_ = np.logspace(7,-2,base=10)
coefs = []
model = lm.Ridge(fit_intercept=False)
mse_test = []
mse_train = []
mse_val = []
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(X_train, y_train)
    yhat_train = model.predict(X_train)
    yhat_val = model.predict(X_val)
    yhat_test = model.predict(X_test)
    mse_train.append(MSE(yhat_train,y_train))
    mse_test.append(MSE(yhat_test, y_test))
    mse_val.append(MSE(yhat_val,y_val))
import matplotlib.pylab as plt
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error Ridge')
ax.plot(alphas_,mse_val,label= 'cross-validation error Ridge')
ax.plot(alphas_,mse_test,label='test error Ridge')
plt.legend(loc=2)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.show()

Analisis de Error para Lasso

In [7]:
alphas_ = np.logspace(8,-2,base=10)  # 8 a 0
coefs = []
model = lm.Lasso(fit_intercept=False, tol =0.0000001)
mse_test = []
mse_train = []
mse_val = []
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(X_train, y_train)
    yhat_train = model.predict(X_train)
    yhat_val = model.predict(X_val)
    yhat_test = model.predict(X_test)
    mse_train.append(MSE(yhat_train,y_train))
    mse_test.append(MSE(yhat_test, y_test))
    mse_val.append(MSE(yhat_val,y_val))
import matplotlib.pylab as plt
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error Lasso')
ax.plot(alphas_,mse_val,label= 'cross-validation error Lasso')
ax.plot(alphas_,mse_test,label='test error Lasso')
plt.legend(loc=2)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.show()

Lasso por si solo

In [3]:
import sklearn.linear_model as lm
#para regresión Lasso
model = lm.RidgeCV(fit_intercept = False)
#alphas_ = np.logspace(7,4, base= 10)
#print "calculando alphas"
#a = best_alpha(alphas_, X_train,y_train, model, X_val,y_val)
#a = 560700 #con 560700 .596520     #probar con menor a 100mil
#model.set_params(alpha=a)
model.fit(X_train,y_train)
score = model.score(X_test, y_test)
print "Score de regresion Lasso: %f , con alpha: %f"%(score,model.alpha_)

Score de regresion Lasso: 0.469955 , con alpha: 22239850.028864


reduccion de caracteristica

In [ ]:
import sklearn.linear_model as lm

model = lm.LinearRegression(fit_intercept = False)
model.fit(X_train,y_train)

def MSE(y,yhat): return np.mean(np.power(y-yhat,2))

def fss(x, y,  x_val, y_val, k = 10000):
    p = x.shape[1]-1
    k = min(p, k)
    remaining = range(0, p)
    selected = [p]
    current_score = 0.0
    best_new_score = 0.0
    while remaining and len(selected)<=k :
        score_candidates = []
        for candidate in remaining:
            model = lm.LinearRegression(fit_intercept=False)
            indexes = selected + [candidate]
            x_train = x[:,indexes]
            
            #calcular error mediante cross validation
            mse_candidate = MSE(model.fit(x_train,y).predict(x_val[:,indexes]) , y_val)
            #mse_candidate = C_V(x_train,y, K=10)
            score_candidates.append((mse_candidate, candidate))
        score_candidates.sort()
        score_candidates[:] = score_candidates[::-1]
        best_new_score, best_candidate = score_candidates.pop() #el de menor error es el mejor candidato
        remaining.remove(best_candidate)
        selected.append(best_candidate)
        print "selected = %s ..."%names_x[best_candidate]
        print "totalvars=%d, mse = %f"%(len(indexes),best_new_score)
    return selected
sequence = fss(X_train,y_train,X_val,y_val)
print sequence

DATASET CON MAS CARACTERISTICAS

In [ ]:
X_test = csr_matrix(mmread('ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/test.x.mm'))
y_test = np.loadtxt('ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/test.y.dat')

X_train = csr_matrix(mmread('ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/train.x.mm'))
y_train = np.loadtxt('ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/train.y.dat')

X_val = csr_matrix(mmread('ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/dev.x.mm'))
y_val = np.loadtxt('ngrams-deprels-origin.runtime.budget.numscreen.ratings.seasons/dev.y.dat')

print "termino de cargar la data"

import sklearn.linear_model as lm

model = lm.LinearRegression(fit_intercept = False)
model.fit(X_train,y_train)
best_score = model.score(X_test,y_test)
print "Score de regresion ordinaria: %f"%(best_score)

model = lm.Ridge(fit_intercept = False )
alphas_ = np.logspace(4,9, base = 10)
a = best_alpha(alphas_,X_train,y_train,model,X_val,y_val)
model.set_params(alpha=a)
model.fit(X_train,y_train)
score = model.score(X_test, y_test)
print "Score de regresion Ridge: %f , con alpha: %f"%(score,a)

model = lm.Lasso(fit_intercept = False ,tol = 0.0000001)
alphas_ = np.logspace(7,4, base= 10)
a = best_alpha(alphas_, X_train,y_train, model, X_val,y_val)  #a = 255954.792270 
model.set_params(alpha=a)
model.fit(X_train,y_train)
score = model.score(X_test, y_test)
print "Score de regresion Lasso: %f , con alpha: %f"%(score,a)

In [ ]:
print "hola"